In [1]:
import pandas as pd
import numpy as np
#import networkx as nx

In [4]:
DATADIR = '../hike/'

In [5]:
train = pd.read_csv(DATADIR + 'train.csv', dtype={"is_chat": np.int8})

In [6]:
test = pd.read_csv(DATADIR + 'test.csv',dtype={"is_chat": np.int8})

In [8]:
df = pd.concat((train.iloc[:,:2], test.iloc[:,1:3]), axis=0)

In [9]:
df.shape

(82438770, 2)

#### node1 and node2 unique identifier to spot the duplicates

In [10]:
a = df[['node1_id', 'node2_id']].min(axis=1).astype(str) +'_'+ df[['node1_id', 'node2_id']].max(axis=1).astype(str)

#### the following series is a variable if any duplicate exists or not

In [11]:
b = a.duplicated()

In [12]:
dup = a[b]

In [13]:
dup.head()

20052    5716426_6436830
24383       91871_902812
26328    5455522_7336435
29824    2093088_8527326
39309    4224172_7321499
dtype: object

In [14]:
df_new = df.copy(deep=True)
df_new = df_new.reset_index(drop=True)

In [15]:
df_new['a'] = a.values
df_new['b'] = b.values
df_new['is_chat']= train.is_chat

In [16]:
df_new.head()

,node1_id,node2_id,a,b,is_chat
0,8446602,6636127,6636127_8446602,False,0.0
1,1430102,7433949,1430102_7433949,False,0.0
2,2803017,8372333,2803017_8372333,False,0.0
3,4529348,894645,894645_4529348,False,0.0
4,5096572,4211638,4211638_5096572,False,0.0


In [17]:
### data frame containing only duplicated enteries
df_new_2 = df_new[df_new.a.isin(dup)]

In [18]:
df_new_2.shape

(25352730, 5)

In [19]:
df_new_2 = df_new_2.sort_values(by='a')

In [20]:
df_new_2.head()

,node1_id,node2_id,a,b,is_chat
37990688,1085652,1000001,1000001_1085652,True,0.0
36654614,1000001,1085652,1000001_1085652,False,0.0
6733508,1000001,1448168,1000001_1448168,False,0.0
65722675,1448168,1000001,1000001_1448168,True,0.0
54414400,1000001,5720020,1000001_5720020,True,0.0


In [21]:
df_new_2.reset_index(drop=True, inplace=True)

In [22]:
df_new_2['flag'] = df_new_2.index%2

In [23]:
df_new_2.head()

,node1_id,node2_id,a,b,is_chat,flag
0,1085652,1000001,1000001_1085652,True,0.0,0
1,1000001,1085652,1000001_1085652,False,0.0,1
2,1000001,1448168,1000001_1448168,False,0.0,0
3,1448168,1000001,1000001_1448168,True,0.0,1
4,1000001,5720020,1000001_5720020,True,0.0,0


### we have a data set with consecutive rows as the reversal pairs, now we just seperate out the even and odd rows and map the values into one another

In [24]:
df_new_2['is_chat_1'] = df_new_2.is_chat

In [25]:
df1, df2 = df_new_2[df_new_2.flag==0], df_new_2[df_new_2.flag==1]

In [26]:
df1.reset_index(drop=True, inplace=True)
df2.reset_index(drop=True, inplace=True)

In [27]:
df1.shape, df2.shape

((12676365, 7), (12676365, 7))

In [28]:
df1.head()

,node1_id,node2_id,a,b,is_chat,flag,is_chat_1
0,1085652,1000001,1000001_1085652,True,0.0,0,0.0
1,1000001,1448168,1000001_1448168,False,0.0,0,0.0
2,1000001,5720020,1000001_5720020,True,0.0,0,0.0
3,1000001,6073560,1000001_6073560,False,0.0,0,0.0
4,1000001,8147194,1000001_8147194,False,0.0,0,0.0


In [29]:
df2.head()

,node1_id,node2_id,a,b,is_chat,flag,is_chat_1
0,1000001,1085652,1000001_1085652,False,0.0,1,0.0
1,1448168,1000001,1000001_1448168,True,0.0,1,0.0
2,5720020,1000001,1000001_5720020,False,0.0,1,0.0
3,6073560,1000001,1000001_6073560,True,0.0,1,0.0
4,8147194,1000001,1000001_8147194,True,0.0,1,0.0


In [30]:
df1['leak_feature']=df2.is_chat_1
df2['leak_feature']=df1.is_chat_1

/home/weightsandbias/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/weightsandbias/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [31]:
df1.tail()

,node1_id,node2_id,a,b,is_chat,flag,is_chat_1,leak_feature
12676360,99,5014626,99_5014626,True,0.0,0,0.0,0.0
12676361,6338478,99,99_6338478,False,1.0,0,1.0,NaN
12676362,6520787,99,99_6520787,True,NaN,0,NaN,0.0
12676363,7810756,99,99_7810756,False,1.0,0,1.0,1.0
12676364,99,8314130,99_8314130,False,0.0,0,0.0,NaN


In [32]:
df2.tail()

,node1_id,node2_id,a,b,is_chat,flag,is_chat_1,leak_feature
12676360,5014626,99,99_5014626,False,0.0,1,0.0,0.0
12676361,99,6338478,99_6338478,True,NaN,1,NaN,1.0
12676362,99,6520787,99_6520787,False,0.0,1,0.0,NaN
12676363,99,7810756,99_7810756,True,1.0,1,1.0,1.0
12676364,8314130,99,99_8314130,True,NaN,1,NaN,0.0


In [33]:
temp=pd.concat((df1[['node1_id','node2_id','is_chat','leak_feature','b']],
           df2[['node1_id','node2_id','is_chat','leak_feature','b']]),axis=0).reset_index(drop=True)

In [34]:
temp.head()

,node1_id,node2_id,is_chat,leak_feature,b
0,1085652,1000001,0.0,0.0,True
1,1000001,1448168,0.0,0.0,False
2,1000001,5720020,0.0,0.0,True
3,1000001,6073560,0.0,0.0,False
4,1000001,8147194,0.0,0.0,False


In [35]:
temp.shape

(25352730, 5)

In [36]:
temp_df=df.merge(temp[['node1_id','node2_id','leak_feature','b']],on=['node1_id','node2_id'],how='left')

In [37]:
temp_df.leak_feature.fillna(-1,inplace=True)
temp_df.b.fillna(False,inplace=True)

In [38]:
temp_df.b=temp_df.b.astype('int8')
temp_df.leak_feature=temp_df.leak_feature.astype('int8')

In [41]:
temp_df[['leak_feature','b']].to_pickle('leak_feature.pkl')

In [46]:
temp_df.leak_feature.value_counts()

-1    60708580
 0    20733674
 1      996516
Name: leak_feature, dtype: int64